<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso2/ciclo1/M5U1_Introducci%C3%B3n_al_Aprendizaje_Profundo_con_Tensorflow_y_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1QqjbbEZ1w7xoawV020Jj_R46PKRi6A_e" alt = "Encabezado MLDS 5" width = "100%">  </img>

#**Introducción al aprendizaje profundo**
----

En este taller guiado haremos un acercamiento a los conceptos prácticos fundamentales para el desarrollo de modelos de **_Deep Learning_** con **_Tensorflow y Keras_**.

Veremos:

- Grafos computacionales
- Elementos de _TensorFlow_:
  - Tensores
  - _Eager execution_
  - _TensorBoard_
  - Derivacion con _TensorFlow_
  - Funciones de Optimizacion
  - Integración con _Numpy_
- Modelos básicos con _Keras_
- Regresión Logística con _TensorFlow_
- Regresión Logística con _Keras_

# **_Tensorflow_**

_Tensorflow_ es una plataforma end-to-end de uso libre para _Machine Learning_ (ML). Se compone de un ecosistema exhaustivo y flexible de herramientas, librerías y recursos de la comunidad que permite a investigadores y desarrolladores construir e implementar aplicaciones basadas en ML, especialmente enfocadas en modelos de aprendizaje profundo (_Deep Learning_).

Unas de las principales ventajas de Tensorflow son:

|Fácil construcción de modelos|Producción robusta de ML|Experimentación para investigación|
|---|---|---|
|<img src="https://drive.google.com/uc?export=view&id=1xbDw0lShlbrk4XKjWsQMyBqU5giW1XWk" alt = "Ejemplo de Tensores de distintos ordenes" width="80%"> | <img src="https://drive.google.com/uc?export=view&id=1AJ_pmzhYP4X6Ixrc82mjYxpKbwIrHpmG" width="80%" />|<img src="https://drive.google.com/uc?export=view&id=1tiqfRraa13DlTA-dJThgWg1rY0WGa3Hh" width="80%" />|
| Facilita la implementación y el entrenamiento <br/> de modelos de ML utilizando APIs de alto nivel <br/> como *Keras* con *eager execution*, lo cual, <br/> permite una fácil construcción y depuración.| Permite entrenar e implementar fácilmente <br/> modelos en la nube, en dispositivos móviles o <br/>en un navegador sin importar el lenguaje usado.| Tiene una arquitectura simple y flexible para <br/> llevar nuevas ideas a experimentación y producción.
|



# **1. Importar Tensorflow** <a class="anchor" id="section1"></a>
---

A continuación importaremos _Tensorflow_.

In [ ]:
#!pip install tensorflow==2.15.0 tensorboard==2.15.0

In [ ]:
import tensorflow as tf

Y luego definiremos algunas funciones para facilitar la visualización. Usaremos estas funciones a lo largo del notebook para comprender los conceptos detrás de _Tensorflow_.

In [ ]:
%load_ext tensorboard
import tensorboard
tensorboard.__version__

In [ ]:
# Función para mostrar regiones de decisión
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

def plot_data(X, y):
    y_unique = np.unique(y)
    colors = plt.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        plt.scatter(this_X[:, 0], this_X[:, 1],  c=np.array([color]),
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    plt.legend(loc="best")
    plt.title("Data")


def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 50)
    y_vals = np.linspace(min_y, max_y, 50)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    vals = [[XX[i, j], YY[i, j]] for i in range(grid_r) for j in range(grid_c)]
    preds = pred_fun(np.array(vals))
    ZZ = np.reshape(preds, (grid_r, grid_c))
    plt.contourf(XX, YY, ZZ, 100, cmap = plt.cm.coolwarm, vmin= -1, vmax=2)
    plt.colorbar()
    CS = plt.contour(XX, YY, ZZ, 100, levels = [0.1*i for i in range(1,10)])
    plt.clabel(CS, inline=1, fontsize=10)
    plt.xlabel("x")
    plt.ylabel("y")


# Eliminar cualquier logs de ejecuciones anteriores
!rm -rf ./logs/

# Configurando logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

**Acelerando el entrenamiento de los Modelos de _Deep learning_**

El aprendizaje profundo o _Deep Learning_ se refiere a una familia de modelos basados en **redes neuronales artificiales** que son escalables (funcionan bien con grandes cantidades de datos) y tienen gran efectividad en diferentes tareas complejas sobre datos de distintas naturalezas.  

Una de las principales características de muchos de los **_frameworks_** de _Deep Learning_ actuales es su habilidad para ser escalados. Esta habilidad es esencial dado que los modelos de _Deep Learning_ :

* Son modelos que tienen una gran cantidad de parámetros por aprender.
* Se aplican sobre conjuntos de datos enormes.

Estas características de los modelos de _Deep Learning_ hacen que un entrenamiento convencional usando solamente una CPU, con varios núcleos, sea extremadamente lento. Por ello se han desarrollado varias estrategias para acelerar dicho entrenamiento. Entre estas se encuentra el uso de **GPU** (_Graphic processing Unit_) o unidades de procesamiento gráfico donde miles de procesadores pueden ejecutar tareas en paralelo acelerando así el entrenamiento.


# **2. Conceptos Generales en _Tensorflow_** <a class="anchor" id="section2"></a>
---
Veamos algunos conceptos básicos de _Tensorflow_:


## 2.1 Tensores

La principal estructura de datos utilizada en _Tensorflow_ son los **tensores**. Se tratan de arreglos multidimensionales que permiten guardar información, además, se pueden entender como una generalización de los escalares, que serían tensores de orden 0 (0D-tensor), vectores o tensores de orden 1 (1D-tensor) y las matrices (2D-tensor). Veamos a continuación algunos ejemplos de tensores de distintos órdenes :

<center><img src="https://drive.google.com/uc?export=view&id=1Mw0prq_XRmLbM6LdNz0isdjp_vQAZAQa" alt = "Ejemplo de Tensores de distintos ordenes" width="80%" /></center>

In [ ]:
# Definimos un 1D-tensor (vector) constante a partir de una lista
t = tf.constant([2, 3, 4, 5], dtype=tf.int32)
print(t)

Un tensor tiene **dos propiedades básicas** :
- Su forma (_shape_).
- Su tipo (_dtype_).

El _shape_, al igual que en _Numpy_, indica el orden, el número de dimensiones y el tamaño de cada dimensión.

In [ ]:
print(t.shape)

En el ejemplo anterior tenemos un tensor de orden 1, es decir una única dimensión, de tamaño 4. Por otro lado, al igual que en cualquier lenguaje de programación, los tensores tienen un tipo de representación interna : ``tf.int32``, ``tf.float32``, ``tf.string``, entre otros. Una correcta selección del tipo de datos puede hacer los códigos **más eficientes**. En el ejemplo anterior, el tipo del tensor es entero de 32 bits.

- El siguiente ejemplo corresponde a un tensor de orden 2, una matriz, cuyo tipo es flotante de 32 bits :

In [ ]:
# Definimos un 2D-tensor (matriz) variable a partir de una lista
t = tf.constant([[9, 5], [1, 0]], dtype=tf.float32)
print(t)

En _Tensorflow_ hay dos tipos principales de tensores :

* ```tf.constant``` : Son arreglos multidimensionales inmutables, es decir, son tensores que no van a cambiar durante la ejecución.
* ```tf.Variable``` : Se trata de tensores cuyos valores pueden cambiar durante la ejecución (por ejemplo, los parámetros de un modelo se definen como variables, ya que, estos valores se actualizan de forma iterativa).

Veamos un ejemplo de variables en _Tensorflow_ :

In [ ]:
# Definimos un 2D-tensor (matriz) variable a partir de una lista
t = tf.Variable([[1, 2], [3, 4]], dtype=tf.float32)
print(t)

In [ ]:
# Al tensor variable le podemos asignar un nuevo valor
t.assign([[-2, -1], [-3, -7]])
print(t)

In [ ]:
# También podemos sumarle o restarle un valor
t.assign_add([[1, 1], [1, 1]])
print(t)
t.assign_sub([[2, 2], [2, 2]])
print(t)

Podemos realizar diversas operaciones y definir funciones sobre tensores. Así mismo, _Tensorflow_ provee un *slicing* similar al de los arreglos de _Numpy_. Veamos un ejemplo :

In [ ]:
# Definimos un 2D-tensor A
A=tf.constant([[1, 2], [3, 4]], dtype=tf.float32)

# Definimos un 2D-tensor B
B=tf.constant([[-1, -2], [-3, -4]], dtype=tf.float32)

In [ ]:
# Suma
A + B

In [ ]:
# Resta
A - B

In [ ]:
# Multiplicación por un escalar (definido en Python)
3 * A

In [ ]:
# Multiplicación elemento a elemento
A * B

In [ ]:
# Multiplicación matricial
print(tf.matmul(A, B))

In [ ]:
# Ejemplos de slicing
print(f'Tensor original:\n {A}')
# Seleccionamos la primera fila
print(f'Primera fila:\n {A[0]}')
# Seleccionamos el primer elemento de la primera fila
print(f'Primer elemento de la primera fila: \n {A[0, 0]}')
# Selecionamos la segunda columna
print(f'Segunda columna:\n {A[:, 1]}')
# Invertimos las filas
print(f'Filas invertidas:\n {A[::-1]}')

También podemos aplicar diferentes funciones matemáticas a todos los elementos de un tensor :

In [ ]:
# Logaritmo
tf.math.log(A)

In [ ]:
# Coseno
tf.math.cos(A)

Otros tipos de operaciones aritméticas, funciones matemáticas y operaciones de álgebra lineal se pueden encontrar en el paquete ```tf.math``` y de álgebra lineal en el paquete ```tf.linalg```.

## 2.2 _Eager execution_

*Tensorflow* provee un ambiente de programación **imperativo** (*Eager execution*) para evaluar las operaciones de forma inmediata sin la necesidad de que el usuario especifique explícitamente un grafo. Es decir, el resultado de las operaciones son valores concretos en lugar de variables simbólicas dentro del grafo computacional. Además, también permite construir el grafo de forma automática en casos donde sea requerido. Esto permite comenzar más fácilmente a programar en *Tensorflow* y a depurar modelos. Adicionalmente, *Eager execution* soporta la mayoría de las funcionalidades de *Tensorflow* y también permite aceleración por GPU.

*Eager execution* es una plataforma flexible para la investigación y la experimentación que provee :

* **Interfaz intuitiva** : Permite desarrollar código de forma natural y usar estructuras de datos de _Python_. También permite desarrollar rápido aplicaciones en casos con modelos pequeños y con  pocos datos.

* **Depuración simple** : Ejecutar las operaciones directamente. Permite revisar a detalle los modelos durante ejecución y evaluar cambios. Además, utiliza herramientas de depuración nativas en _Python_ para reportar errores de forma inmediata.

* **Control natural** : Controlar las variables desde _Python_ en lugar de un control por medio de un grafo, simplifica la especificación de modelos más dinámicos.

La versión 2.0 de *Tensorflow* trae por defecto *Eager execution*.

**Importante: esto quiere decir que todas las operaciones matemáticas que hicimos en la sección anterior, se hicieron bajo ejecución *eager*.**

In [ ]:
# Revisamos la versión de Tensorflow
tf.__version__

In [ ]:
# Revisamos si eager execution está activa
tf.executing_eagerly()

Por defecto, *Eager execution* ejecuta las operaciones de **forma secuencial**, es decir, no construye un grafo computacional de no ser que sea necesario para alguna operación o de ser especificado. Para que _Tensorflow_ construya el grafo debemos utilizar el decorador ```tf.function``` como se muestra a continuación :

In [ ]:
# Definimos una función normal en Python (ejecuta las operaciones de forma secuencial)
def my_func1(x):
    a = tf.constant(10, tf.float32, name= 'a')
    b = tf.constant(-5, tf.float32, name= 'b')
    c = tf.constant(4, tf.float32, name= 'c')
    y = a * x * x + b * x + c
    return y

print(type(my_func1))
print(my_func1(5))

In [ ]:
# Definimos una función decorada (construye internamente el grafo computacional)
@tf.function
def my_func2(x):
    a = tf.constant(10, tf.float32, name= 'a')
    b = tf.constant(-5, tf.float32, name= 'b')
    c = tf.constant(4, tf.float32, name= 'c')
    y = a * x * x + b * x + c
    return y

print(type(my_func2))
print(my_func2(5))

<center>
<img src="https://drive.google.com/uc?export=view&id=14r8LaMThJRvIS7s0gpA9o0X29Rm743ZB" alt = "Gráfica de la función 10x^2-5^x+4" width="40%" />
</center>

Note que `my_func1` y `my_func2` corresponden a la misma función :

$$f(x)=10x^2-5x+4$$

Ahora veamos una comparación entre el tiempo promedio de cálculo de estas dos funciones :

In [ ]:
%%timeit -n 1000
my_func1(tf.constant([1, 2, 3, 4], dtype=tf.float32))

In [ ]:
%%timeit -n 1000
my_func2(tf.constant([1, 2, 3, 4], dtype=tf.float32))

## 2.3 Grafo Computacional y _TensorBoard_

_Tensorflow_ es de gran utilidad para el _Deep Learning_ debido fundamentalmente a que permite realizar diferenciación automática y paralelización de operaciones matemáticas. _Tensorflow_ consigue esto al construir internamente un **grafo computacional** :

<center>
<img src="https://drive.google.com/uc?export=view&id=1YXiVhjZ5N3WrEBCM081Rhn6MhSTVlXVm" alt = "Ejemplo de Grafo Computacional"  width="40%" />
</center>

Este grafo define un flujo de datos basado en expresiones matemáticas. Más específicamente, _Tensorflow_ utiliza un grafo dirigido donde cada nodo representa una operación.

- Una de las principales ventajas de usar un grafo computacional es que las operaciones se definen como relaciones o dependencias, lo cual permite que los cómputos sean fácilmente simplificados y paralelizados. Esto es mucho más práctico en comparación con un programa convencional donde las operaciones se ejecutan de forma secuencial.

- _**TensorBoard**_ es una herramienta que proporciona las mediciones y visualizaciones necesarias durante el flujo de trabajo del _Machine Learning_. Permite realizar un seguimiento de las métricas de los experimentos, como la pérdida y la precisión, visualizar el gráfico del modelo, proyectar incrustaciones a un espacio de menor dimensión y mucho más.

A continuación, usaremos _Tensorboard_ para visualizar el gráfico de una función. La visualización es interactiva, por lo que puedes expandir y colapsar nodos, acercarte, alejarte, desplazarte, etc.

Generalmente TensorBoard se usa para monitorear el entrenamiento de un modelo, como lo veremos más adelante en el curso. El siguiente código monitorea la ejecución de la función `my_func2` definida anteriormente.

In [ ]:
# tf.summary.trace_on() y tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=logdir)
# Llamando a una tf.function: my_func2
x = tf.constant(5, tf.float32)
z = my_func2(x)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0)

In [ ]:
%tensorboard --logdir logs/func

## 2.4 Diferenciación

<center>
<img src="https://drive.google.com/uc?export=view&id=1uGkaY7xq4XubB0BmcrIQlsgtBf7P3ngo" alt= "Gráfico de Back Propagation" width="60%" />
</center>

Podemos calcular el gradiente de cualquier expresión con respecto a una variable que aparezca en ella. Para ello tenemos que usar un objeto `GradientTape` que lleva la cuenta de las operaciones ejecutadas dentro del contexto correspondiente, luego , utilizar ese registro y los gradientes de las operaciones para calcular el gradiente respectivo utilizando una forma general de *backpropagation* llamada **diferenciación en modo inverso** :


In [ ]:
x = tf.constant(5.0)
with tf.GradientTape() as t:
    t.watch(x)
    y = my_func2(x)
dy_dx = t.gradient(y, x)
dy_dx

Esto funciona tanto para *Eager execution* como para los grafos :


In [ ]:
x = tf.constant(5.0)
with tf.GradientTape() as t:
    t.watch(x)
    y = my_func1(x)
dy_dx = t.gradient(y, x)
dy_dx

Podemos utilizar el gradiente descendente para encontrar un mínimo del polinomio representado en el gráfico. Aquí tenemos que cambiar `x` para que sea una variable dado que su valor se va a actualizar para diferentes evaluaciones sucesivas del grafo.

In [ ]:
x = tf.Variable(5.0)
dy_dx = 1
while np.abs(dy_dx) > 0.01:
    with tf.GradientTape() as t:
        t.watch(x)
        y = my_func1(x)
    dy_dx = t.gradient(y, x)
    x.assign(x - 0.01*dy_dx)
    print("f(x)=", my_func1(x).numpy(), "df(x)/dx=", dy_dx.numpy())

Note que $3.375001$ es una muy buena aproximación al valor mínimo real que puede obtener la función `my_func1`: $3.375$.

<center>
<img src="https://drive.google.com/uc?export=view&id=1stkjv1_Wi-I0KhRt8DuoE5sRUR47bDUw" alt = "Ejemplo de gradiente Descendente" width="50%" />
</center>



## 2.5 Optimizadores

_Tensorflow_ ya tiene implementado el proceso de gradiente descendente: [`tf.keras.optimizers.SGD`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD). El gradiente descendente, o gradiente descendente estocástico (_Stochastic Gradient Descent, SGD_) es un *optimizador*, que nos permitirá optimizar (valga la redundancia) la función que definirá nuestro modelo de _Machine Learning_.

A continuación, optimizáremos la misma función del caso anterior, pero usaremos directamente `tf.keras.optimizers.SGD` para el proceso iterativo. Nótese que definimos `learning_rate=0.01`, esto es, fijar la tasa de aprendizaje, o el tamaño de paso, en $0.01$ veces la magnitud del gradiente, igual a como lo hicimos en el caso anterior.


- `loss`: La función de pérdida que se quiere optimizar.
- `var_list`: La lista de las variables respecto a las cuales se deben calcular los gradientes.

`apply_gradients` reemplaza `minimize`. Este método toma una lista de pares (gradiente, variable) y actualiza las variables según los gradientes calculados.

Veamos cómo hacerlo:

- Primero definimos el optimizador e inicializamos la variable `x`:


In [ ]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01)

- Definimos la función `create_f_x`, que crea una variable de TensorFlow `x` y una función `f_x` (correspondiente a $f(x)=10x^2-5x+4$), que depende de `x` y será nuestra función a optimizar :

In [ ]:
def create_f_x(init_val):
    x = tf.Variable(init_val)
    def f_x():
        return 10 * x * x - 5 * x + 4
    return x, f_x

- Hacemos 40 iteraciones de gradiente descendente con la función `minimize`:

In [ ]:
x, f_x = create_f_x(5.0)
for i in range(40):
    with tf.GradientTape() as tape:
        loss = f_x()
    gradients = tape.gradient(loss, [x])
    sgd.apply_gradients(zip(gradients, [x]))
    print('f(x)=', f_x().numpy())

De nuevo, llegamos a un valor óptimo aproximado de $3.375$. `tf.keras.optimizers.SGD` no es el único optimizador que ofrece _Tensorflow_. Entre los más populares por su eficiencia y robustez están :
- **_Adam_** ([`tf.keras.optimizers.Adam
`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam))
- **_RMSprop_** ([`tf.keras.optimizers.RMSprop`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/RMSprop)).

En general, el mejor optimizador va a depender de cada modelo, de los datos y de la  exploración que se realice. La lista completa de optimizadores disponibles la puedes encontrar en :
- https://www.tensorflow.org/api_docs/python/tf/keras/optimizers.

A continuación repetiremos la optimización del caso anterior pero, usaremos `tf.keras.optimizers.RMSprop` en lugar de `SGD`, y una tasa de aprendizaje de $0.5$ en lugar de $0.01$.

In [ ]:
rms = tf.keras.optimizers.RMSprop(learning_rate=0.5)
x, f_x = create_f_x(5.)

for i in range(20):
    with tf.GradientTape() as tape:
        loss = f_x()
    gradients = tape.gradient(loss, [x])
    rms.apply_gradients(zip(gradients, [x]))
    print('f(x)=', f_x().numpy())

Note la diferencia en el número de iteraciones necesarias para llegar al valor óptimo de la función.

## 2.6 Integración de _TensorFlow_ con _Numpy_

Una de las principales ventajas de _Tensorflow 2.0_ es su compatibilidad con arreglos y operaciones de _Numpy_. Esta última es la librería de álgebra lineal más usada en _Python_.

- Veamos algunos ejemplos con _Numpy_ y _Tensorflow_:

In [ ]:
# Definimos un arreglo en numpy, la función linspace crea una secuencia de 'num' números
# Igualmente distanciados entre dos límites 'start' y 'stop'
x = np.linspace(start=0, stop=1, num=10)
print(x)

In [ ]:
# Realizamos algunas operaciones en Tensorflow, por ejemplo, sumar todos los números en el tensor
acum = tf.reduce_sum(x)
print(acum)

In [ ]:
# Definimos un tensor en Tensorflow
x = tf.linspace(0.0,1.0,10)
print(x)

In [ ]:
# Realizamos algunas operaciones en Numpy
acum = np.sum(x)
print(acum)

In [ ]:
# Convertir arreglo de Numpy a tensor
x = np.linspace(0,1,10)
t = tf.constant(x)
print(t)

In [ ]:
# Convertir tensor a arreglo de Numpy
t = tf.linspace(0.0,1.0,10)
x = t.numpy()
print(x)

## 2.7 CPU vs GPU

En general la sintaxis de _Tensorflow_ es muy similar a la de _Numpy_, y dada esta funcionalidad, podemos usar arreglos de cualquiera de los dos _frameworks_ conjuntamente. Sin embargo, esto no es recomendable: es necesario tener en cuenta que _Numpy_ está diseñado para trabajar con la CPU, mientras que _Tensorflow_ permite utilizar hardware para acelerar el cómputo (e.g., unidades de procesamiento gráfico (GPU) o unidades de procesamiento tensorial (TPU)). La conversión de información entre la vRAM de la GPU y la RAM del computador es una operación costosa, y solo se debe considerarse en caso de que sea completamente necesaria.

- Veamos un ejemplo de cómo seleccionar el tipo de hardware que se usará y una comparación del desempeño.

**_En el siguiente código especificamos que los cálculos deben hacerse utilizando la CPU_** :


In [ ]:
%%timeit
# Ejecución en CPU
with tf.device("CPU:0"):
    X = tf.random.uniform(shape=(1000, 1000), minval=0, maxval=1)
    # La siguiente función evalúa si un tensor está en determinado dispositivo
    assert X.device.endswith("CPU:0")
    tf.matmul(X, X)

**_El siguiente código especifica que el dispositivo para llevar a cabo la ejecución debe ser la GPU_** :

In [ ]:
%%timeit
# Ejecución en GPU
with tf.device("GPU:0"):
    X = tf.random.uniform(shape=(1000, 1000), minval=0, maxval=1)
    # La siguiente función evalúa si un tensor está en determinado dispositivo
    assert X.device.endswith("GPU:0")
    tf.matmul(X, X)

Ahora, veamos una comparación de la misma operación en _Numpy_ :

In [ ]:
%%timeit
# Ejecución en CPU con numpy
X = np.random.uniform(0, 1, size=(1000, 1000))
X @ X

Note la diferencia de tiempo de todas las ejeciciones anteriores. En todos los casos el cálculo es el mismo, pero **el proceso de _Numpy_ es considerablemente más demorado** que el proceso de _Tensorflow_ en GPU.

# **3. Modelo lineal en Tensorflow** <a class="anchor" id="section3"></a>
---
Veamos ejemplos de implementación de modelos básicos de *Machine Learning* usando tanto _Tensorflow_ como _Keras_. **Primero, _Tensorflow_** :


## 3.1 Cargar datos

Implementaremos un modelo de regresión logística para clasificar los siguientes datos :

In [ ]:
from sklearn.datasets import make_blobs

X_train, Y_train = make_blobs(n_samples=100, centers=2, n_features=2, random_state=0)
plt.figure(figsize=(8, 6))
plot_data(X_train, Y_train)

In [ ]:
# Volvemos tensores los datos de entrenamiento:

X_t = tf.constant(X_train, dtype=tf.float32)
Y_t = tf.constant(Y_train, dtype=tf.float32)

# Añadimos una dimensión a las etiquetas, para que tengan shape=(100,1)
Y_t = tf.expand_dims(Y_t, axis=-1, name=None)

## 3.2 Definir el modelo

<center>
<img src="https://drive.google.com/uc?export=view&id=1C9z4WHUTm5DCENCYHp0jstnHv-BT1ezy" alt = "Regresión Lineal" width="60%" />
</center>

La regresión logística es un **modelo de clasificación binaria**, es decir, las etiquetas pueden tomar el valor de 0 o 1. El modelo logístico se muestra a continuación :

$$
\tilde{y_i}=\frac{1}{1+e^{-w_1 x_1-w_2 x_2-\dots-w_m x_m -w_0}}=\frac{1}{1+e^{-\vec{w}\cdot\vec{x}_i}},
$$

donde $\vec{w}=(w_1, w_2, \dots, w_m, w_0)$ y $\vec{x}_i=(x_1,x_2,\dots,x_m, 1)^T$. Este es un modelo lineal que se compone de dos parámetros: los pesos ($w_1,w_2,\dots,w_m$) y el sesgo o intercepto ($w_0$). La expresión $\vec{x}_i$ corresponde a la representación de la muestra `i`-ésima de nuestro conjunto de datos, y $\tilde{y_i}$ es la predicción que hace el modelo para la muestra $\vec{x}_i$. Si tenemos en cuenta que

$$
\text{sigmoid}(z)=\frac{1}{1+e^{-z}}
$$

El modelo de regresión logística equivalente a la siguiente expresión :

$$
\tilde{y_i}=\text{sigmoid}(-\vec{w}\cdot\vec{x}_i)
$$

Usando funciones de _Tensorflow_ podemos implementar la función sigmoide. Entonces podemos facilmente definir nuestra función de predicción, o modelo :

In [ ]:
def log_reg(w, b, X):
    return 1/(1+tf.math.exp(-(tf.matmul(X, w) + b)))

Asignando unos valores aleatorios a los pesos $(w,b)$, por ejemplo $w=(0.5,0.5)$ y $b=-0.5$, podemos calcular algunas predicciones con el modelo no entrenado :

In [ ]:
X_train[:10, :]

In [ ]:
# Calculamos las predicciones sobre las primeras 10 muestras de X_train:
log_reg(w=np.array([[0.5],[0.5]]),
        b=-0.5,
        X=X_train[:10, :])

Aproximemos las predicciones al entero más cercano para obtener resultados binarios :

reshape(-1), hace el ejercicio de reduccion

In [ ]:
np.rint(log_reg(w=np.array([[0.5],[0.5]]),
        b=-0.5,
        X=X_train[:10, :])).reshape(-1)

Como todas las predicciones eran valores mayores a `0.5`, todas son hacia la clase `1`. Como estas predicciones se calculan con un modelo no entrenado, seguramente no van a ser muy acertadas. Compare el vector de predicciones anterior con el vector de etiquetas reales correspondiente:

In [ ]:
print(Y_train[0:10])

## 3.3 Función de pérdida

<center>
<img src="https://drive.google.com/uc?export=view&id=1jRb2TAHdC78rRT29igyn-KONSNnWkEYO" alt = "Gráfica de Función de perdida" width="80%" />
</center>


La regresión logística se puede abordar como un problema de optimización con una **función de pérdida** (o *Loss Function*) conocida como **entropía cruzada binaria** (*binary cross-entropy*), la cual se muestra a continuación :

$$L(\vec{w}, y, \tilde{y} )=-\frac{1}{N}\sum_{i=1}^{N}[y_i\log(\tilde{y}_i)+(1-y_i)\log(1-\tilde{y}_i)],$$

donde $N$ es el número de datos de entrenamiento, $y_i$ representa el valor real de la clase (la etiqueta 0 o 1) y $\tilde{y}_i$ el valor que predice el modelo. Esta función cuantifica el error de predicción del modelo, y por tanto se hace pequeña en la medida que el modelo haga predicciones correctas. Por ejemplo, si $y_i=0$ y $\tilde{y}_i\approx0$ entonces $(y_i\log(\tilde{y}_i)+(1-y_i)\log(1-\tilde{y}_i))\approx0$, es decir, penaliza poco. Por el contrario, si $y_i=1$ y $\tilde{y}_i\approx0$ entonces $(y_i\log(\tilde{y}_i)+(1-y_i)\log(1-\tilde{y}_i))$ es mucho mayor a 0, es decir, penaliza bastante. Algo analogo sucede cuando $y_i=1$ y $\tilde{y}_i\approx1$ o $\tilde{y}_i\approx0$, respectivamente.


Nuestro objetivo será ajustar los parámetros $\vec{w}$ del modelo $\text{sigmoid}(-\vec{w}\cdot\vec{x}_i)$ para que $L(\vec{w})$ sea lo más cercano a cero posible.

<center>
<img src="https://drive.google.com/uc?export=view&id=1g1EolYPXm-0HTIqhhIlN7r8z4OXb-HYK" alt = "Visualización de pérdidas" width="80%" />
</center>

A continuación definiremos la función `bce` para calcular el *binary cross-entropy* a partir de las etiquetas reales `y_true` y las que predice el modelo `y_pred`.

In [ ]:
# Definimos la función binary cross-entropy
def bce(y_true, y_pred):
    return -tf.math.reduce_mean(y_true * tf.math.log(y_pred) + (1. - y_true) * tf.math.log(1. - y_pred))

Usando `bce` podemos entonces definir la función de pérdida `loss_fun` que vamos a optimizar en el entrenamiento:

In [ ]:
# Definimos una función que crea las variables representando los parámetros del
# modelo, aplica el modelo y calcula la función de pérdida
def create_model(X_t, Y_t):
    w = tf.Variable([[1.0],[-1.0]])
    b = tf.Variable(0.5)
    def loss_fun():
        y_pred = log_reg(w, b, X_t)
        return bce(Y_t, y_pred)
    return w, b, loss_fun

## 3.4 Entrenamiento

Ahora definimos las operaciones para calcular la pérdida y optimizarla usando `tf.keras.optimizers.SGD` con `learning_rate=0.5` :

In [ ]:
# Creamos nuestro modelos a optimizar

w, b, loss_fun = create_model(X_t, Y_t)

# Stochastic Gradient Descent como optimizador:

opt = tf.keras.optimizers.SGD(learning_rate=0.5)

# Iteramos 50 veces el proceso de optimización del gradiente descendente,
# y guardamos el valor final de la función de pérdida en cada iteración:

losses = []
for i in range(50):
    print('Epoch',i,end=' ')
    with tf.GradientTape() as tape:
        loss = loss_fun()
    gradients = tape.gradient(loss, [w, b])
    opt.apply_gradients(zip(gradients, [w, b]))
    print('loss =',loss_fun().numpy())
    losses.append(loss_fun().numpy())

Podemos visualizar el comportamiento de la función de pérdida a medida que avanzan las iteraciones :

In [ ]:
plt.figure(figsize = (8,16/3))
plt.title('Loss Function vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(losses)

Tal como queremos, la función de pérdida disminuye a medida que las iteraciones (o **_epochs_**) avanzan, es decir, estamos optimizando correctamente la función de pérdida de nuestro modelo, y está *aprendiendo*.

## 3.5 Resultados

Ahora utilizamos la función `plot_decision_function`, definida al principio del notebook, para visualizar la región de decisión.  

In [ ]:
def pred_fun(X):
    X = tf.constant(X, dtype=tf.float32)
    return log_reg(w, b, X).numpy()

plt.figure(figsize = (8,16/3))
plot_decision_region(X_train, pred_fun)
plot_data(X_train, Y_train)

# **4. Modelo lineal en _Keras_** <a class="anchor" id="section4"></a>
---
Ahora haremos de nuevo una regresión logística, usando los mismos datos, pero armando el modelo en _Keras_ para notar las diferencias con _Tensorflow_:


## 4.1 _Keras_
Originalmente, _Keras_ era un *framework* de alto nivel escrito en _Python_ que utilizaba diferentes *backends* de *Deep learning* como: _Tensorflow_, _CNTK_ o _Theano_. Actualmente, es un paquete dentro de _Tensorflow 2.0_ que nos permite simplificar tanto el entrenamiento como el diseño de modelos de *Machine Learning* y redes neuronales.

```tf.keras``` es usado para la creación rápida de modelos y tiene tres ventajas :

* **Amigable al usuario**: _Keras_ tiene una interfaz simple y consistente que ha sido optimizada para el uso en casos típicos.
* **Modular**: La construcción de modelos se basa en conectar bloques personalizables con pocas restricciones.
* **Fácil extensión**: Permite implementar nuevos módulos fácilmente usando todas las funcionalidades de _Tensorflow_, lo cual, facilita la construcción de nuevos modelos o modelos del estado del arte.

## 4.2 Definir el modelo

Veamos la implementación de este modelo en _Keras_. Primero definimos un objeto `tf.keras.models.Sequential`:

In [ ]:
# Definimos el modelo
model = tf.keras.models.Sequential()

Luego, usamos una capa `tf.keras.layers.Dense`, que nos permite fácilmente aplicar un operador linear a las entradas del modelo y luego aplicar una función sigmoide, entendida aquí como **función de activación**. `tf.keras.layers.Dense` es en sí misma una función que recibe una serie de argumentos para definir correctamente la capa de neuronas del modelo :

*   `units`: Un número entero que define el número de neuronas de la capa.
*   `input_shape`: Una tupla con la diménsión de los datos de entrada.
*   `activation`: Una función de activación que se aplica a la salida de la capa.

En este caso podemos usar la función sigmoide que ofrece _Tensorflow_: `tf.keras.activations.sigmoid`



In [ ]:
model.add(tf.keras.layers.Dense(
                units=1,              # Definimos el tamaño de la dimensión de la predicción, en nuestro caso 1.
                input_shape=(2, ),    # Definimos el tamaño de la dimensión de los datos de entrada, en nuestro caso 2.
                activation=tf.keras.activations.sigmoid   # Definimos la función sigmoid que actua sobre w.x
                ))

## 4.3 Compilar el modelo

El proceso de aprendizaje se configura con la función `compile`. Los argumentos necesarios para el entrenamiento posterior son:

*   `optimizer`: Función que define el optimizador de gradiente descendente que se aplicará sobre la función de pérdida.  
*   `loss`: Función de pérdida que define el objetivo de optimización.

Aquí utilizaremos la función de pérdida de entropía cruzada `bce` que definimos anteriormente y un optimizador `tf.optimizers.SGD`. Este optimizador está definido como una función que automáticamente realiza el proceso de la función `minimize` que vimos en la sección anterior, y recibe como parámetro la tasa de aprendizaje o `learning_rate`.

Además, si queremos, el proceso de aprendizaje puede obtener un seguimiento de la exactitud o *accuracy* de las predicciones del modelo. Para esto usamos otro argumento de `compile`:

*   `metrics`: Una lista de métricas de desempeño que serán evaluadas durante entrenamiento y prueba.

Podemos usar entonces cualquier métrica de desempeño correctamente definida. Por ejemplo, el *accuracy* o exactitud. Nosotros usaremos `tf.keras.metrics.BinaryAccuracy` que mide la exactitud en aplicaciones de clasificación binaria cuando la salida del modelo es una sola neurona. Entonces nuestra función `compile` queda así:

In [ ]:
# Compilamos el modelo
model.compile(
              optimizer=tf.optimizers.SGD(learning_rate=0.5), # Definimos el optimizador y la tasa de aprendizaje
              loss=bce,                                       # Definimos la función de pérdida
              metrics = [tf.keras.metrics.BinaryAccuracy()]   # Definimos una métrica para hacer seguimiento del desempeño durante el entrenamiento
              )

La estructura del modelo se puede visualizar con `summary()`:

In [ ]:
model.summary()

Y también podemos obtener una vizualicación capa a capa con `tf.keras.utils.plot_model`, función que recibe como argumento a el modelo `model`:

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

Note que efectivamente tenemos un modelo con tres parámetros por aprender. Inicialmente, _Keras_ genera los pesos del modelo de forma aleatoria (los cuales se estimarán durante el entrenamiento), veamos un ejemplo de esto:

In [ ]:
def pred_fun(X):
    X = tf.constant(X, dtype=tf.float32)
    return model.predict(X, verbose=0)

plt.figure(figsize = (8,16/3))
plot_decision_region(X_train, pred_fun)
plot_data(X_train, Y_train)

Note que el modelo no está separando correctamente las clases. Ahora sí, vamos a entrenarlo:

## 4.4 Entrenamiento

El modelo se entrena llamando a la función `fit`. Esta función necesita los siguientes argumentos :


*   `x`: Un array de _Numpy_ o tensor de _Tensorflow_ con los datos usados para entrenar.
*   `y`: Un array de _Numpy_ o tensor de _Tensorflow_ con las etiquetas de los datos usados para entrenar.
*   `epochs`: Un número entero, correspondiente al número de iteraciones completas de entrenamiento sobre el conjunto de datos `x`.


En este punto pasamos como argumentos los datos de entrenamiento `X_t` y `Y_t` y el número de iteraciones de optimización que queremos hacer. Dese cuenta de que en este caso **no tenemos que ser explícitos** con las operaciones del cálculo del gradiente ni de la actualización de las variables.

In [ ]:
model.fit(x=X_t,        # Datos de entrada para entrenar
          y=Y_t,        # Etiquetas (salida) para entrenar
          epochs=50     # Número de iteraciones
          )

Note que `fit` por defecto nos va mostrando en cada iteración o *epoch* el valor de la función de pérdida *loss* y el desempeño según la métrica que especificamos en la compilación. Este seguimiento del entrenamiento se puede modificar con el parámetro `verbose` de la función `fit`.

## 4.5 Resultados

El modelo entrenado puede utilizarse para clasificar nuevas muestras
utilizando `predict`. Esta función recibe como argumento los datos sobre los cuales queremos obtener una predicción:

In [ ]:
print(model.predict(np.array([[0, 0]])))

Esta es la región de decisión del modelo entrenado :

In [ ]:
def pred_fun(X):
    X = tf.constant(X, dtype=tf.float32)
    return model.predict(X, verbose=0)

plt.figure(figsize = (8,16/3))
plot_decision_region(X_train, pred_fun)
plot_data(X_train, Y_train)

El modelo lineal mejora mucho después de que se entrena; separa correctamente las clases.

Con esto terminamos una introducción completa a los elementos necesarios para trabajar con modelos _Tensorflow_ y _Keras_.

¡Muchas gracias!

# **Recursos adicionales**
---
Los siguientes enlaces corresponden a sitios en donde encontrará información muy útil para profundizar en el conocimiento de las funcionalidades de la librería *TensorFlow* y *Keras*:

- [*TensorFlow - Tutorials*](https://www.tensorflow.org/tutorials)
- [*TensorFlow - Basics*](https://www.tensorflow.org/guide/basics)
- [*TensorFlow - Keras*](https://www.tensorflow.org/guide/keras/sequential_model)
* _Origen de los íconos_
    - Tensorflow. Model [SVG]. https://www.tensorflow.org/site-assets/images/marketing/home/model.svg
    - Tensorflow. Robust [SVG]. https://www.tensorflow.org/site-assets/images/marketing/home/robust.svg
    - Tensorflow. Research [SVG]. https://www.tensorflow.org/site-assets/images/marketing/home/research.svg

# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
  * [Juan Sebastián Lara](https://http://juselara.com/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](https://www.linkedin.com/in/mario-andres-rodriguez-triana-394806145/).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*